# crop_spectra_files
Jupyter notebook for cropping LV0.NC files created by rpgpy from the *.LV0 binaries. This notebook is used for selecting a time-height region of interest to obtain smaller-sized files for PEAKO training

In [6]:
import sys
import xarray as xr
import glob
from numba import jit
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
from rpgpy import read_rpg

In [7]:
# find LV0.NC files in current directory
rpgpy_files = glob.glob('/srv/data/shareddata/hyytiala/24*.LV0.NC')

In [8]:
rpgpy_files = [name for name in rpgpy_files if '_cropped_' not in name]
rpgpy_files.sort()
print(rpgpy_files)

[]


In [56]:
click = [None,None]
release = [None,None]

def line_select_callback(eclick, erelease):
    click[:] = eclick.xdata, eclick.ydata
    release[:] = erelease.xdata, erelease.ydata

In [57]:

@jit(nopython=True, fastmath=True)
def radar_moment_calculation(signal, vel_bins, DoppRes):
    """
    Calculation of radar moments: reflectivity, mean Doppler velocity, spectral width,
    skewness, and kurtosis of one Doppler spectrum. Optimized for the use of Numba.

    Note:
        Divide the signal_sum by 2 because vertical and horizontal channel are added.
        Subtract half of of the Doppler resolution from mean Doppler velocity, because

    Args:
        - signal (float array): detected signal from a Doppler spectrum
        - vel_bins (float array): extracted velocity bins of the signal (same length as signal)
        - DoppRes (int): resolution of the Doppler spectra (different for each chirp)

    Returns:
        dict containing

        - **Ze_lin** (*float array*): reflectivity (0.Mom) over range of velocity bins [mm6/m3]
        - **VEL** (*float array*): mean velocity (1.Mom) over range of velocity bins [m/s]
        - **sw** (*float array*): spectrum width (2.Mom) over range of velocity bins [m/s]
        - **skew** (*float array*): skewness (3.Mom) over range of velocity bins
        - **kurt** (*float array*): kurtosis (4.Mom) over range of velocity bins
    """

    signal_sum = np.sum(signal)  # linear full spectrum Ze [mm^6/m^3], scalar
    Ze_lin = signal_sum / 2.0
    pwr_nrm = signal / signal_sum  # determine normalized power (NOT normalized by Vdop bins)

    VEL = np.sum(vel_bins * pwr_nrm)
    vel_diff = vel_bins - VEL
    vel_diff2 = vel_diff * vel_diff
    sw = np.sqrt(np.abs(np.sum(pwr_nrm * vel_diff2)))
    sw2 = sw * sw
    skew = np.sum(pwr_nrm * vel_diff * vel_diff2 / (sw * sw2))
    kurt = np.sum(pwr_nrm * vel_diff2 * vel_diff2 / (sw2 * sw2))
    VEL = VEL - DoppRes / 2.0

    return Ze_lin, VEL, sw, skew, kurt


def spectra2moments(spec_data, **kwargs):
    """
    This routine calculates the radar moments: reflectivity, mean Doppler velocity, spectrum width, skewness and
    kurtosis from the level 0 spectrum files of the 94 GHz RPG cloud radar.

    Args:
        spec_data
    Returns:
        moments
    """

    # initialize variables:
    n_ts, n_rg, n_vel = spec_data['doppler_spectrum'].shape
    n_chirps = len(spec_data.chirp)
    Z = np.full((n_ts, n_rg), np.nan)
    V = np.full((n_ts, n_rg), np.nan)
    SW = np.full((n_ts, n_rg), np.nan)
    SK = np.full((n_ts, n_rg), np.nan)
    K = np.full((n_ts, n_rg), np.nan)

    spec_lin = spec_data['doppler_spectrum'].values.copy()
    mask = (spec_lin <= 0.0) | (np.isnan(spec_lin))
    print('masked values:', np.sum(mask))
    spec_lin[mask] = 0.0

    ## combine the mask for "contains signal" with "signal has more than 1 spectral line"
    #mask1 = np.all(mask, axis=2)
    #mask2 = ZSpec['edges'][:, :, 1] - ZSpec['edges'][:, :, 0] <= 0
    #mask3 = ZSpec['edges'][:, :, 1] - ZSpec['edges'][:, :, 0] >= n_vel
    #mask = mask1 * mask2 * mask3
    chirp_indices = np.append(spec_data.chirp_start_indices.values, len(spec_data.range))
    for iC in range(n_chirps):
        Doppler_resolution = np.nanmedian(np.diff(spec_data.velocity_vectors[iC].values))
        for iR in range(chirp_indices[iC], chirp_indices[iC + 1]):  # range dimension
            for iT in range(n_ts):  # time dimension
                if np.all(mask[iT, iR,:]): continue
                #lb, rb = ZSpec['edges'][iT, iR, :]
                Z[iT, iR], V[iT, iR], SW[iT, iR], SK[iT, iR], K[iT, iR] = \
                    radar_moment_calculation(spec_lin[iT, iR, :], spec_data.velocity_vectors[iC].values, Doppler_resolution)


    moments = {'Ze': Z, 'VEL': V, 'sw': SW, 'skew': SK, 'kurt': K}
    # create the mask where invalid values have been encountered
    invalid_mask = np.full((spec_data.doppler_spectrum.values.shape[:2]), True)
    invalid_mask[np.where(Z > 0.0)] = False
    
    # mask invalid values with fill_value = -999.0
    for mom in moments.keys():
        moments[mom][invalid_mask] = -999.0

    ## build larda containers from calculated moments
    #container_dict = {mom: make_container_from_spectra([ZSpec], moments[mom], paraminfo[mom], invalid_mask, 'VHSpec') for mom in moments.keys()}

    return moments



In [58]:
# function definitions
@jit(nopython=True, fastmath=True)
def estimate_noise_hs74(spectrum, navg=1, std_div=6.0, nnoise_min=1):
    """REFERENCE TO ARM PYART GITHUB REPO: https://github.com/ARM-DOE/pyart/blob/master/pyart/util/hildebrand_sekhon.py

    Estimate noise parameters of a Doppler spectrum.
    Use the method of estimating the noise level in Doppler spectra outlined
    by Hildebrand and Sehkon, 1974.

    Args:
        spectrum (array): Doppler spectrum in linear units.
        navg (int, optional): The number of spectral bins over which a moving average
            has been taken. Corresponds to the **p** variable from equation 9 of the article.
            The default value of 1 is appropriate when no moving average has been applied to the spectrum.
        std_div (float, optional): Number of standard deviations above mean noise floor to specify the
            signal threshold, default: threshold=mean_noise + 6*std(mean_noise)
        nnoise_min (int, optional): Minimum number of noise samples to consider the estimation valid.

    Returns:
        tuple with

        - **mean** (*float*): Mean of points in the spectrum identified as noise.
        - **threshold** (*float*): Threshold separating noise from signal. The point in the spectrum with
          this value or below should be considered as noise, above this value
          signal. It is possible that all points in the spectrum are identified
          as noise. If a peak is required for moment calculation then the point
          with this value should be considered as signal.
        - **var** (*float*): Variance of the points in the spectrum identified as noise.
        - **nnoise** (*int*): Number of noise points in the spectrum.

    References:
        P. H. Hildebrand and R. S. Sekhon, Objective Determination of the Noise
        Level in Doppler Spectra. Journal of Applied Meteorology, 1974, 13, 808-811.
    """
    sorted_spectrum = np.sort(spectrum)
    nnoise = len(spectrum)  # default to all points in the spectrum as noise

    rtest = 1 + 1 / navg
    sum1 = 0.
    sum2 = 0.
    for i, pwr in enumerate(sorted_spectrum):
        npts = i + 1
        if npts < nnoise_min:
            continue

        sum1 += pwr
        sum2 += pwr * pwr

        if npts * sum2 < sum1 * sum1 * rtest:
            nnoise = npts
        else:
            # partial spectrum no longer has characteristics of white noise.
            sum1 -= pwr
            sum2 -= pwr * pwr
            break

    mean = sum1 / nnoise
    var = sum2 / nnoise - mean * mean

    threshold = mean + np.sqrt(var) * std_div

    return mean, threshold, var, nnoise

In [59]:
# select file index
i = 0

In [60]:
spec_data = xr.open_dataset(rpgpy_files[i])
print('file opened:', rpgpy_files[i])

file opened: PEAKO spectra files/240114_080000_P09_ZEN.LV0.NC


In [61]:
spec_data.time

<xarray.DataArray 'time' (time: 1118)>
array([726912000, 726912003, 726912006, ..., 726915591, 726915594, 726915597],
      dtype=int32)
Coordinates:
  * time     (time) int32 726912000 726912003 726912006 ... 726915594 726915597
Attributes:
    long_name:        Time of Sample
    units:            s
    comment:          since 1.1.2001
    rpg_manual_name:  Time

In [62]:
moments = spectra2moments(spec_data)

masked values: 405209383


In [65]:
matplotlib.use('TkAgg')
fig, ax = plt.subplots(1)
ax.pcolormesh(10*np.log10(moments['Ze'].T))
ax.set_title(rpgpy_files[i].split('/')[-1])
props = dict(facecolor='blue', alpha=0.5)
rect = RectangleSelector(ax, line_select_callback, interactive=True,
                                  props=props)

fig.show()



/var/folders/c6/_f246r9d34x5bd4fll64h_2r0000gn/T/ipykernel_87538/615295551.py:3: RuntimeWarning: invalid value encountered in log10
  ax.pcolormesh(10*np.log10(moments['Ze'].T))


In [66]:
x1, y1 = [int(c) for c in click]
x2, y2 = [int(r) for r in release]

In [67]:
x1, x2

(304, 450)

In [68]:
mask = np.zeros(moments['Ze'].shape)
mask[x1:x2+1, y1:y2+1] = 1
mask_3D = np.tile(mask[:,:,np.newaxis], spec_data.doppler_spectrum.shape[2])

In [69]:
print('masking spec data')
a = spec_data.doppler_spectrum.where(mask_3D>0)
a = a.isel(time=range(x1, x2+1))


masking spec data


In [70]:
## if noise compression was off #
## i.e. if the radar Doppler spectra saved with noise floor, no noise threshold factor was set in the radar software 
## then we need to compute the noise floor and crop the spectra for PEAKO

#print('H&S noise estimation')
#threshold = np.apply_along_axis(estimate_noise_hs74, 2, a)[:, :, 1]
#print('removing noise below threshold...')
#a = a.where(a > threshold[:,:,np.newaxis])
#

In [71]:
new_spec_data = xr.Dataset({'doppler_spectrum':a, 'velocity_vectors': spec_data.velocity_vectors, 
                            'chirp_start_indices': spec_data.chirp_start_indices,
                           'range_layers': spec_data.range_layers,
                           'n_range_layers': spec_data.n_range_layers})

In [72]:
filename_out = rpgpy_files[i][:-7]+f'_cropped_x{x1}_{x2}_y{y1}_{y2}.LV0.NC'
print('filename out:', filename_out)

filename out: PEAKO spectra files/240114_080000_P09_ZEN_cropped_x304_450_y1_160.LV0.NC


In [73]:
new_spec_data.to_netcdf(filename_out)